In [ ]:
import pandas as pd
import numpy as np
import random
import csv
from matplotlib import pyplot as plt
import seaborn as sns
import category_encoders as ce
import warnings
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')

# 1. Kodowanie zmiennych kategorycznych

Wczytuję dane i zmieniam wszystkie litery na małe

In [ ]:
allegro=pd.read_csv("allegro-api-transactions.csv")
allegro["it_location"]=allegro["it_location"].apply(str.lower)

# TargetEncoding

In [ ]:
target=ce.TargetEncoder()
allegro["it_location_targetencoding"]=target.fit_transform(allegro["it_location"],allegro["price"])
allegro["it_location_targetencoding"].head()

Jak widać dla dla każdej kateogorii została przypisana wartość liczbowa. Gdybyśmy użyli One-hot encodingu powstałyby 7903 nowe kolumny, ponieważ tyle mamy unikalnych kategorii i z tego względu Target Encoding wydaje się być lepszy. Z drugiej strony zamiana na pojedyńczą wartość numeryczną wprowadza pewnego rodzaju uporządkowanie i daje możliwość porównywania, której nie było w pierwotnych danych.

TargetEncoding przypisuje każdej z kategorii, średnią wartość zmiennej objaśnianej dla tej kategorii.

In [ ]:
allegro[["main_category"]].describe()

# OneHotEncoding

In [ ]:
onehot=ce.OneHotEncoder()
onehot_df=pd.read_csv("allegro-api-transactions.csv")
onehot_df=onehot_df.join(onehot.fit_transform(allegro["main_category"]))
onehot_df.head()

OneHotEncoding stworzył 27 nowych kolum odpowiadających 27 unikalnym kategoriom z "main_category". Nie mamy teraz problemu ze storzeniem sztucznego porządku podczas enncodingu dzięki czemu model nie będzie ich porównywał tak jak w przypadku gdy zastąpiliśmy kategorię jedną wartością.

# OrdinalEncoding

In [ ]:
ordinal=ce.OrdinalEncoder()
ordinal_df=pd.read_csv("allegro-api-transactions.csv")
ordinal_df["main_category_ordinal"]=ordinal.fit_transform(allegro["main_category"])
ordinal_df["main_category_ordinal"].head()

OrdinalEncoding działa na zasadzie przyporządkowania każdej kategorii losowej liczby całkowitej.Użycie tego pozostawia problem tworzenia sztucznego pożądku.

# HashingEncoding

In [ ]:
hashing=ce.HashingEncoder()
hashing_df=pd.read_csv("allegro-api-transactions.csv")
hashing_df=hashing_df.join(hashing.fit_transform(allegro["main_category"]))
hashing_df.head()

HashingEncoding zamienia każdą kategorię na n-wymiarowy wektor wypełniony 0 i 1. Mając kontrolę nad n możemy uniknąć robienia wielu kolum jak w onehot ale narażamy się na to, że dwie kategoie będą miały tego samego hasha.

# 2. Uzupełnianie braków

In [ ]:
random.seed(11111)
imputer=KNNImputer(n_neighbors=2, weights="uniform")
allegro2=pd.read_csv("allegro-api-transactions.csv")[["price", "it_seller_rating", "it_quantity"]]
allegro2=allegro2[0:42000]
RMSE=[0 for i in range(10)]
for i in range(10):
    remove_index=random.sample(range(0,42000),4200)
    allegro2_removed=allegro2.copy()
    allegro2_removed.iloc[remove_index,1]=None
    allegro2_imputed=pd.DataFrame(imputer.fit_transform(allegro2_removed),columns=["price", "it_seller_rating", "it_quantity"])
    RMSE[i]=mean_squared_error(allegro2["it_seller_rating"],allegro2_imputed["it_seller_rating"],squared=False)
print("RMSE: ",RMSE)
print("Std: ",np.std(RMSE))


In [ ]:
RMSE1=[0 for i in range(10)]
RMSE2=[0 for i in range(10)]
for i in range(10):
    remove_index1=random.sample(range(0,42000),4200)
    remove_index2=random.sample(range(0,42000),4200)
    allegro2_removed=allegro2.copy()
    allegro2_removed.iloc[remove_index1,1]=None
    allegro2_removed.iloc[remove_index2,2]=None
    allegro2_imputed=pd.DataFrame(imputer.fit_transform(allegro2_removed),columns=["price", "it_seller_rating", "it_quantity"])
    RMSE1[i]=mean_squared_error(allegro2["it_seller_rating"],allegro2_imputed["it_seller_rating"],squared=False)
    RMSE2[i]=mean_squared_error(allegro2["it_quantity"],allegro2_imputed["it_quantity"],squared=False)
print("RMSE it_seller_rating: ",RMSE1)
print("RMSE it_quantity: ",RMSE2)
print("Std RMSE it_seller_rating: ",np.std(RMSE1))
print("Std RMSE it_quantity: ",np.std(RMSE2))

In [ ]:
d={'RMSE it_seller_rating':RMSE+RMSE1,'iteracja':[0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9],'col2':["RMSE z jedną niepełną kolumną" for i in range(10)]+["RMSE z dwoma niepełnymi kolumnami" for i in range(10)]}
df=pd.DataFrame(data=d)
fig=plt.figure(figsize=(12,12))
g=sns.barplot(x='iteracja',y='RMSE it_seller_rating',hue='col2',data=df)
g.legend_.set_title(None)

Z powyższego wykresu wyraźnie widać, że po usunięciu 10% wartości z innej kolumny algorytm imputacji działa zauważalnie  gorzej. Używanie automatycznego wypełniania braków np. KNNimutation wiąże się z dużym błędem oraz odchyleniem standardowym, rosnącym wraz z ilością kolumn, w których te braki występują.